# Web Scrapper

- Obtener resultados de google
- Obtener los párrafos de cada resultado
- Preprocesar y guardar

In [1]:
from util.scrapper import *
from models.lda_utils import get_keywords
import re
is_well_formed_link = re.compile(r'^https?://.+/.+$')

def is_useful(string):
    return string != '\n'

keywords = get_keywords()
data = []
for keyword in keywords:
    try:
        results = scrape_google(keyword, 3, "es")
        for result in results:
            data.append(result)
    except Exception as e:
        print(e)
    finally:
        time.sleep(10)


links = [result['link'] for result in data if is_well_formed_link.match(result['link'])]



In [2]:
from models.lda_utils import get_keywords
get_keywords()

['estrategia precio red coste control caso usuario copia positivo preguntar',
 'experiencia cliente masivo valor sacrificio oferta individuo necesidad campo tema',
 'cliente experiencia valor masivo negocio necesidad diferencia ciberespacio misma precio',
 'cliente marca social comunidad conectividad horizontal mundo precio red concepto',
 'emergente conducta organismo comportamiento complejo agente desarrollar superior regla complejas',
 'plataforma conocimiento red fuente dato colaborativa valor com modelo humano',
 'adoptar innovador individuo consumidor modelo gente pc grupo rol coeficiente',
 'rifkin industrial infraestructura marginal actual ley decir sociedad factura capitalista',
 'coste tarea global compartir concepto favorecer ley coase emergencia capacidad',
 'largo cola ofrecer venta regla cliente demanda nicho oferta minorista']

In [3]:
links

['https://br.escueladenegociosydireccion.com/business/marketing-ventas/estrategias-de-precios-en-un-mercado-competitivo/',
 'https://es.semrush.com/blog/estrategia-de-precios/',
 'http://diposit.ub.edu/dspace/bitstream/2445/66261/1/TFM_MOI_Pedrero-Yolanda-jun2015.pdf',
 'http://tdx.cat/bitstream/handle/10803/396345/TD_2016_TenaMonferrer.pdf?sequence=1&isAllowed=y',
 'https://www.tesisenred.net/bitstream/handle/10803/146251/David_Rodriguez_Rabad%C3%A1n%20Benito.pdf?sequence=1&isAllowed=y',
 'https://www.tesisenred.net/bitstream/handle/10803/385276/tesdoc_a2016_roman_david_analisis_promociones.pdf?sequence=1&isAllowed=y',
 'https://rockcontent.com/es/blog/marketing-digital/',
 'https://core.ac.uk/download/pdf/44310136.pdf',
 'https://e-archivo.uc3m.es/bitstream/handle/10016/22498/rosario_rivera_tesis.pdf',
 'https://www.vocesenelfenix.com/content/la-%E2%80%9Crevoluci%C3%B3n%E2%80%9D-de-las-redes-sociales-sociedad-educaci%C3%B3n-y-nueva-profesi%C3%B3n',
 'https://dialnet.unirioja.es/desca

In [4]:
paragraphs = {}

In [5]:
import urllib.request
from bs4 import BeautifulSoup

for link in links:
    try:
        content = urllib.request.urlopen(link)
        read_content = content.read()
        soup = BeautifulSoup(read_content,'html.parser')
        pAll = soup.find_all('p')
        paragraphs[link]= list(filter(is_useful, [p.text for p in pAll]))
    except:
        pass

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHA

In [6]:
def is_useful(p):
    return len(p) > 3

for key in paragraphs.keys():
    paragraphs[key] = list(filter(is_useful, paragraphs[key]))

In [7]:
from document import Document
preprocess_list = ['lemmatized_string', 'sentences', 'preprocessed_sentences', 'paragraphs', 'preprocessed_paragraphs']
documents = []

for key in paragraphs.keys():
    p = paragraphs[key]
    if len(p) != 0:
        doc = Document()
        doc.from_paragraphs(paragraphs = paragraphs[key], preprocess_list=preprocess_list)
        doc.title = key
        documents.append(doc)


In [8]:
def process_file(document):
    return [document.title, document.title, document.string, document.word_count(), document.type_count(), document.sentences,\
                       document.lemmatized_string, document.stemmed_string, document.simple_preprocessed_string, document.topic, \
                       document.named_entities, document.bigrams, document.trigrams, document.lemmatized_bigrams, \
                       document.lemmatized_trigrams, document.stemmed_bigrams, document.stemmed_trigrams, \
                       document.simple_preprocessed_bigrams, document.simple_preprocessed_trigrams, document.preprocessed_sentences,
                       document.paragraphs, document.preprocessed_paragraphs, document.urls]

scrapped = [process_file(doc) for doc in documents]

In [9]:
import pandas as pd
dataframe_scrapped = pd.DataFrame(data = scrapped, columns=['original_path', 'document_title', 'string', 'word_count', 'type_count', 'sentences','lemmatized_text', 'stemmed_text', 'simple_preprocessed', 'topic', 'named_entities', 'tokens_bigrams', 'tokens_trigrams', 'lemmatized_bigrams', 'lemmatized_trigrams', 'stemmed_bigrams', 'stemmed_trigrams', 'simple_preprocessed_bigrams', 'simple_preprocessed_trigrams', 'preprocessed_sentences', 'paragraphs', 'preprocessed_paragraphs', 'urls'])
dataframe_scrapped.to_csv('../data/scrapped.csv')